In [6]:
import requests

base_url = 'https://r.jina.ai/'
link = 'https://www.myprotein.com.sg/nutrition/protein.list'
num = 1
page_number = f'?page_number={num}'
url = f'{base_url}{link}{page_number}'
headers = {
    'Accept': 'application/json'
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    data = response.json()  # Parse the JSON response
    print(data)
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

{'code': 200, 'status': 20000, 'data': {'title': 'Myprotein Singapore', 'url': 'https://www.myprotein.com.sg/nutrition/protein.list?page_number=1', 'content': 'Protein Shakes, Powders & Supplements | MYPROTEIN™\n===============\n                                                                                                               \n\n[Skip to main content](https://www.myprotein.com.sg/nutrition/protein.list?page_number=1#mainContent)\n\nsg - SGD Change\n\n*   [Contact us](https://www.myprotein.com.sg/customer-services/contact-us.list)\n*   [Help](https://www.myprotein.com.sg/help-centre.list)\n\n[Myprotein Singapore](https://www.myprotein.com.sg/)\n\n[Myprotein Singapore](https://www.myprotein.com.sg/)\n\nTry Our Search \n\n[Account](https://www.myprotein.com.sg/accountHome.account)\n\n*   [Login](https://www.myprotein.com.sg/accountHome.account)\n*   [Register](https://www.myprotein.com.sg/accountCreate.account)\n*   [Wishlist](https://www.myprotein.com.sg/wishlists.account)\n

In [8]:
import json
with open('save.json', 'w') as f:
    json.dump(data, f, indent=4)

## Scraping all possible product in the category
This is done per page, link to a theme

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Fetch the HTML content
url = 'https://www.myprotein.com.sg/nutrition/protein.list'  # Replace with your target URL
response = requests.get(url)
html_content = response.text

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the main product list container
product_list = soup.find('ul', class_='productListProducts_products')

# Initialize a list to hold product details
products = []
main_theme = soup.find('h1', class_='responsiveProductListHeader_title')

# Extract product details
for product in product_list.find_all('li', class_='productListProducts_product'):
    product_details = {}
    # Extract product name
    product_name_tag = product.find('h3', class_='productBlock_productName')
    if product_name_tag:
        product_details['name'] = product_name_tag.text.strip()
    
    # Extract product URL
    product_link_tag = product.find('a', class_='productBlock_link')
    if product_link_tag:
        product_relative_url = product_link_tag['href']
        product_details['url'] = urljoin(url, product_relative_url)
        
        # Extract the product's unique identifier from the URL
        product_details['data_option_id'] = product_relative_url.split('/')[-1].split('.')[0]
    
    # Extract product price
    product_price_tag = product.find('div', class_='productBlock_fromValue')
    if product_price_tag:
        product_details['price'] = product_price_tag.text.strip()
    
    # Add to products list
    products.append(product_details)

# Print the extracted product details
print(f"Main Theme: {main_theme.text.strip()}")
for product in products:
    print(f"Product Name: {product.get('name')}")
    print(f"Product URL: {product.get('url')}")
    print(f"Product Data Option ID: {product.get('data_option_id')}")
    print(f"Product Price: {product.get('price')}")
    print("-" * 40)
    

Main Theme: Protein
Product Name: Impact Whey Protein Powder
Product URL: https://www.myprotein.com.sg/sports-nutrition/impact-whey-protein-powder/10530943.html
Product Data Option ID: 10530943
Product Price: None
----------------------------------------
Product Name: Impact Whey Isolate Powder
Product URL: https://www.myprotein.com.sg/sports-nutrition/impact-whey-isolate-powder/10530911.html
Product Data Option ID: 10530911
Product Price: None
----------------------------------------
Product Name: Clear Whey Protein Powder
Product URL: https://www.myprotein.com.sg/sports-nutrition/clear-whey-protein-powder/12081395.html
Product Data Option ID: 12081395
Product Price: None
----------------------------------------
Product Name: Clear Vegan Protein
Product URL: https://www.myprotein.com.sg/sports-nutrition/clear-vegan-protein/12360400.html
Product Data Option ID: 12360400
Product Price: None
----------------------------------------
Product Name: Impact Vegan Protein
Product URL: https://

## Themes
- Getting a possible themes and categories from the website
- Inserting the themes and categories into the database


In [23]:
import requests
import json
from bs4 import BeautifulSoup

base_url = 'https://www.myprotein.com.sg'
# Fetch the HTML content
url = 'https://www.myprotein.com.sg/nutrition/protein.list'  # Replace with your target URL
response = requests.get(url)
html_content = response.text

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the main product list container
product_list = soup.find('ul', class_='productListProducts_products')

# Initialize a list to hold product details
products = []
main_theme = soup.find('h1', class_='responsiveProductListHeader_title').text.strip()

# Extract product details
for product in product_list.find_all('li', class_='productListProducts_product'):
    product_details = {}
    # Extract product name
    product_name_tag = product.find('h3', class_='productBlock_productName')
    if product_name_tag:
        product_details['name'] = product_name_tag.text.strip()
    
    # Extract product URL
    product_link_tag = product.find('a', class_='productBlock_link')
    if product_link_tag:
        product_details['url'] = base_url + product_link_tag['href']
    
    # Extract product price
    product_price_tag = product.find('div', class_='productBlock_fromValue')
    if product_price_tag:
        product_details['price'] = product_price_tag.text.strip()
    
    # Add to products list
    products.append(product_details)

# Create a dictionary with the main theme and products
output = {
    "Main Theme": main_theme,
    "Products": products
}

# Save the output as a JSON file
with open('products.json', 'w') as json_file:
    json.dump(output, json_file, indent=4)

print(f"Data saved to products.json")

Data saved to products.json


In [19]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

# Base URL
base_url = "https://www.myprotein.com.sg"

# Function to extract categories and their links recursively
def extract_categories_and_links(soup):
    categories = defaultdict(dict)

    for level_two_item in soup.find_all('li', class_='responsiveFlyoutMenu_levelTwoItem'):
        level_two_link = level_two_item.find('a', class_='responsiveFlyoutMenu_levelTwoLink')
        if level_two_link:
            level_two_name = level_two_link.get('data-context', level_two_link.text.strip())
            level_two_url = level_two_link['href']
            if level_two_url.startswith('/'):
                level_two_url = base_url + level_two_url
            categories[level_two_name] = level_two_url

            # Extract subcategories
            subcategory_list = level_two_item.find('ul', class_='responsiveFlyoutMenu_levelThree')
            if subcategory_list:
                subcategories = {}
                for level_three_item in subcategory_list.find_all('li', class_='responsiveFlyoutMenu_levelThreeItem'):
                    level_three_link = level_three_item.find('a', class_='responsiveFlyoutMenu_levelThreeLink')
                    if level_three_link:
                        level_three_name = level_three_link.get('data-context', level_three_link.text.strip())
                        level_three_url = level_three_link['href']
                        if level_three_url.startswith('/'):
                            level_three_url = base_url + level_three_url
                        subcategories[level_three_name] = level_three_url
                
                if subcategories:
                    categories[level_two_name] = subcategories

    return categories

# Fetch the HTML content from the website
url = 'https://www.myprotein.com.sg/nutrition/protein.list?page_number=1'
response = requests.get(url)
html_content = response.text

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Extract categories and links
category_data = extract_categories_and_links(soup)

# Print the result
for category, subcategories in category_data.items():
    print(f"{category}:")
    if isinstance(subcategories, dict):
        for subcategory, url in subcategories.items():
            print(f"  {subcategory}: {url}")
    else:
        print(f"  {subcategories}")




Protein Powders:
  Protein Powders: https://www.myprotein.com.sg/nutrition/protein.list
  All Protein: https://www.myprotein.com.sg/nutrition/protein.list
  Whey Protein: https://www.myprotein.com.sg/nutrition/protein/whey-protein.list
  Collagen Protein: https://www.myprotein.com.sg/sports-nutrition/collagen-protein/11067704.html
  Clear Protein: https://www.myprotein.com.sg/clear-protein.list
  Protein Isolate: https://www.myprotein.com.sg/nutrition/protein/protein-isolate.list
  Gainer Protein: https://www.myprotein.com.sg/nutrition/weight-management/weight-gainers.list
  Diet Protein: https://www.myprotein.com.sg/nutrition/diet-protein.list
  Vegan Protein: https://www.myprotein.com.sg/nutrition/protein/vegan-protein.list
Protein Bars/Snacks:
  Protein Bars/Snacks: https://www.myprotein.com.sg/nutrition/healthy-food-drinks.list
  All Protein Bars: https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-bars.list
  Snacks: https://www.myprotein.com.sg/nutrition/healthy-fo

In [27]:
from dotenv import load_dotenv
import os
from supabase import create_client, Client
load_dotenv('../../.env.local')

# Replace these with your actual Supabase project URL and public API key
supabase_url = os.getenv('NEXT_PUBLIC_SUPABASE_URL')
supabase_key = os.getenv('NEXT_PUBLIC_SUPABASE_ANON_KEY')

supabase: Client = create_client(supabase_url, supabase_key)

# Define the URLs and categories
categories_urls = {
    "Protein Powders": {
        "Protein Powders": "https://www.myprotein.com.sg/nutrition/protein.list",
        "All Protein": "https://www.myprotein.com.sg/nutrition/protein.list",
        "Whey Protein": "https://www.myprotein.com.sg/nutrition/protein/whey-protein.list",
        "Collagen Protein": "https://www.myprotein.com.sg/sports-nutrition/collagen-protein/11067704.html",
        "Clear Protein": "https://www.myprotein.com.sg/clear-protein.list",
        "Protein Isolate": "https://www.myprotein.com.sg/nutrition/protein/protein-isolate.list",
        "Gainer Protein": "https://www.myprotein.com.sg/nutrition/weight-management/weight-gainers.list",
        "Diet Protein": "https://www.myprotein.com.sg/nutrition/diet-protein.list",
        "Vegan Protein": "https://www.myprotein.com.sg/nutrition/protein/vegan-protein.list"
    },
    "Protein Bars/Snacks": {
        "Protein Bars/Snacks": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks.list",
        "All Protein Bars": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-bars.list",
        "Snacks": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-snacks.list",
        "Protein Cookies": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-cookies.list",
        "Layered Family": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/layered-bar.list"
    },
    "Protein Food": {
        "Protein Food": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-foods.list",
        "Breakfast": "https://www.myprotein.com.sg/weekly-offer/breakfast.list",
        "Nut Butters": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/nut-butters.list"
    },
    "Discover": {
        "Discover": "https://www.myprotein.com.sg/nutrition.list",
        "Best Sellers": "https://www.myprotein.com.sg/nutrition/sg-bestsellers.list",
        "New Products": "https://www.myprotein.com.sg/nutrition/new-in.list",
        "Samples": "https://www.myprotein.com.sg/nutrition/samples.list",
        "Bundles": "https://www.myprotein.com.sg/nutrition/bundles.list",
        "Impact Range": "https://www.myprotein.com.sg/ranges/impact.list",
        "PRO Range": "https://www.myprotein.com.sg/ranges/pro/products.list",
        "New To Supplements?": "https://www.myprotein.com.sg/goal-selector.list"
    },
    "Creatine": {
        "Creatine": "https://www.myprotein.com.sg/nutrition/creatine.list",
        "Creatine Monohydrate": "https://www.myprotein.com.sg/nutrition/creatine/creatine-monohydrate.list"
    },
    "Pre Workout": {
        "Pre Workout": "https://www.myprotein.com.sg/nutrition/pre-post-workout/pre-workout.list"
    },
    "Amino Acids": {
        "Amino Acids": "https://www.myprotein.com.sg/nutrition/amino-acids.list",
        "BCAA Supplements": "https://www.myprotein.com.sg/nutrition/amino-acids/bcaa.list",
        "EAA Supplements": "https://www.myprotein.com.sg/nutrition/amino-acids/eaa.list",
        "Glutamine Supplements": "https://www.myprotein.com.sg/nutrition/amino-acids/glutamine.list",
        "L-Carnitine Supplements": "https://www.myprotein.com.sg/nutrition/amino-acids/l-carnitine.list"
    },
    "Recovery": {
        "Recovery": "https://www.myprotein.com.sg/your-goals/recovery.list",
        "Intra-Workout": "https://www.myprotein.com.sg/nutrition/pre-post-workout/intra-workout.list",
        "Post-Workout": "https://www.myprotein.com.sg/nutrition/pre-post-workout/post-workout.list"
    },
    "Energy": {
        "Energy": "https://www.myprotein.com.sg/nutrition/carbohydrates.list",
        "Energy Supplements": "https://www.myprotein.com.sg/nutrition/carbohydrates/energy-supplements.list",
        "Energy Bars": "https://www.myprotein.com.sg/nutrition/carbohydrates/energy-bars.list",
        "Energy Gels": "https://www.myprotein.com.sg/nutrition/carbohydrates/energy-gels.list"
    },
    "Weight Management": {
        "Weight Management": "https://www.myprotein.com.sg/ranges/myvitamins/weight-loss.list",
        "Thermo": "https://www.myprotein.com.sg/ranges/myvitamins/weight-loss/thermo.list"
    },
    "Accessories": {
        "Accessories": "https://www.myprotein.com.sg/clothing/soft-accessories.list",
        "Training Equipment": "https://www.myprotein.com.sg/nutrition/accessories/weightlifting-equipment.list",
        "Bags": "https://www.myprotein.com.sg/nutrition/accessories/gym-bags.list",
        "Headwear & Gloves": "https://www.myprotein.com.sg/nutrition/accessories/hats-gloves.list",
        "Bottles & Shakers": "https://www.myprotein.com.sg/nutrition/accessories/bottles-shakers.list",
        "Underwear & Socks": "https://www.myprotein.com.sg/nutrition/accessories/underwear-socks.list",
        "Towels": "https://www.myprotein.com.sg/clothing/towels.list",
        "Sliders": "https://www.myprotein.com.sg/clothing/soft-accessories/sliders.list"
    },
    "Protein Bars & Snacks": {
        "Protein Bars & Snacks": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks.list",
        "All Protein Bars": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-bars.list",
        "Protein Cookies": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-cookies.list",
        "Vegan Protein Bars": "https://www.myprotein.com.sg/dietary-needs/vegan/snacks.list"
    },
    "Food": {
        "Food": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/foods.list",
        "Flavourings & Sweeteners": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/flavouring-sweeteners.list",
        "Dried Fruit & Nuts": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/dried-fruit-nuts-snacks.list",
        "Protein Sauces, Syrups & Spreads": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/protein-sauces-syrups-spreads.list"
    },
    "Meal Replacement": {
        "Meal Replacement": "https://www.myprotein.com.sg/nutrition/healthy-food-drinks/meal-replacement.list"
    },
    "Women's Clothing": {
        "Women's Clothing": "https://www.myprotein.com.sg/clothing/womens/view-the-whole-range.list",
        "New In": "https://www.myprotein.com.sg/clothing/womens/new-in.list",
        "Coats & Jackets": "https://www.myprotein.com.sg/clothing/womens/jackets-gilets.list",
        "Hoodies": "https://www.myprotein.com.sg/clothing/womens/hoodies-sweatshirts.list",
        "T-Shirts & Tops": "https://www.myprotein.com.sg/clothing/womens/all-tops.list",
        "Crop Tops": "https://www.myprotein.com.sg/clothing/womens/crop-tops.list",
        "Vests & Tanks": "https://www.myprotein.com.sg/clothing/womens/vests-tanks.list",
        "Sports Bras": "https://www.myprotein.com.sg/clothing/womens/sports-bras.list",
        "Leggings": "https://www.myprotein.com.sg/clothing/womens/gym-leggings.list",
        "Shorts": "https://www.myprotein.com.sg/clothing/womens/gym-shorts.list",
        "Underwear & Socks": "https://www.myprotein.com.sg/clothing/womens/underwear-socks.list",
        "Joggers & Bottoms": "https://www.myprotein.com.sg/clothing/womens/jogging-bottoms.list",
        "Swimwear": "https://www.myprotein.com.sg/clothing/womens/swimwear.list",
        "Maternity": "https://www.myprotein.com.sg/clothing/collections/mama/products.list"
    },
    "Men's Clothing": {
        "Men's Clothing": "https://www.myprotein.com.sg/clothing/mens/view-the-whole-range.list",
        "New In": "https://www.myprotein.com.sg/clothing/mens/new-in.list",
        "Coats & Jackets": "https://www.myprotein.com.sg/clothing/mens/jackets-gilets.list",
        "Hoodies": "https://www.myprotein.com.sg/clothing/mens/sweatshirts-and-hoodies.list",
        "T-Shirts & Tops": "https://www.myprotein.com.sg/clothing/mens/all-tops.list",
        "Stringers & Tanks": "https://www.myprotein.com.sg/clothing/mens/tops-and-vests.list",
        "Joggers & Bottoms": "https://www.myprotein.com.sg/clothing/mens/jogging-bottoms.list",
        "Shorts": "https://www.myprotein.com.sg/clothing/mens/all-shorts.list",
        "Base Layers": "https://www.myprotein.com.sg/clothing/collections/base-layer.list",
        "Underwear & Socks": "https://www.myprotein.com.sg/clothing/mens/underwear-socks.list",
        "Swim Shorts": "https://www.myprotein.com.sg/clothing/mens/swim-shorts.list",
        "Tracksuits": "https://www.myprotein.com.sg/clothing/mens/mens-tracksuits.list"
    },
    "Shop by Activity": {
        "Shop by Activity": "https://www.myprotein.com.sg/clothing/soft-accessories.list",
        "Gym Training": "https://www.myprotein.com.sg/clothing/collections/gym-training.list",
        "Run": "https://www.myprotein.com.sg/clothing/collections/run.list",
        "Functional Training": "https://www.myprotein.com.sg/clothing/collections/functional-training.list",
        "HIIT": "https://www.myprotein.com.sg/clothing/collections/hiit.list",
        "Yoga": "https://www.myprotein.com.sg/clothing/collections/yoga.list",
        "Rest Day": "https://www.myprotein.com.sg/clothing/collections/rest-day/products.list"
    },
    "Outlet | Up to 60% off": {
        "Outlet | Up to 60% off": "https://www.myprotein.com.sg/clothing/sale.list",
        "Men's Outlet": "https://www.myprotein.com.sg/clothing/sale/mens-clothing-sale.list",
        "Women's Outlet": "https://www.myprotein.com.sg/clothing/sale/womens-clothing-sale.list",
        "Accessories Outlet": "https://www.myprotein.com.sg/clothing/accessories-sale.list"
    },
    "Performance": {
        "Performance": "https://www.myprotein.com.sg/ranges/myvitamins/performance.list",
        "Omega 3 & Fish Oils": "https://www.myprotein.com.sg/ranges/myvitamins/omega-3.list",
        "Amino Acids": "https://www.myprotein.com.sg/ranges/myvitamins/amino-acids.list",
        "Recovery & Joint Health": "https://www.myprotein.com.sg/ranges/myvitamins/recovery-joint-health.list",
        "Collagen": "https://www.myprotein.com.sg/nutrition/vitamins-minerals/collagen.list",
        "Pre-Workout": "https://www.myprotein.com.sg/ranges/myvitamins/preworkout.list",
        "Post-Workout": "https://www.myprotein.com.sg/ranges/myvitamins/postworkout.list"
    },
    "Essentials": {
        "Essentials": "https://www.myprotein.com.sg/ranges/myvitamins/essentials.list",
        "Multivitamins": "https://www.myprotein.com.sg/nutrition/vitamins-minerals/multivitamin-supplements.list",
        "Vitamins & Minerals": "https://www.myprotein.com.sg/nutrition/vitamins-minerals.list"
    },
    "Gummies": {
        "Gummies": "https://www.myprotein.com.sg/vitamin-gummies-range.list"
    },
    "Wellness": {
        "Wellness": "https://www.myprotein.com.sg/ranges/myvitamins/wellbeing.list",
        "Bones & Joints": "https://www.myprotein.com.sg/ranges/myvitamins/joints.list",
        "Energy & Focus": "https://www.myprotein.com.sg/ranges/myvitamins/energy-focus.list",
        "Sleep & Relaxation": "https://www.myprotein.com.sg/ranges/myvitamins/sleep-relaxation.list",
        "Men's Health": "https://www.myprotein.com.sg/ranges/myvitamins/mens-health.list",
        "Women's Health": "https://www.myprotein.com.sg/ranges/myvitamins/womens-health.list",
        "Beauty": "https://www.myprotein.com.sg/ranges/myvitamins/beauty.list",
        "Digestion": "https://www.myprotein.com.sg/ranges/myvitamins/digestion.list"
    },
    "Other": {
        "Other": "https://www.myprotein.com.sg/ranges/myvitamins/others.list",
        "Vegan": "https://www.myprotein.com.sg/dietary-needs/vegan/vitamins.list"
    },
    "Protein": {
        "Protein": "https://www.myprotein.com.sg/nutrition/protein/vegan-protein.list",
        "Vegan Protein Powder": "https://www.myprotein.com.sg/nutrition/protein/vegan-protein.list"
    },
    "Vitamins & Minerals": {
        "Vitamins & Minerals": "https://www.myprotein.com.sg/dietary-needs/vegan/vitamins.list",
        "Vegan Omega": "https://www.myprotein.com.sg/ranges/myvegan/vegan-omega.list",
        "Vegan Tablets & Capsules": "https://www.myprotein.com.sg/ranges/myvegan/vegan-tablets-capsules.list",
        "Vegan Gummies": "https://www.myprotein.com.sg/ranges/myvegan/vegan-gummies.list"
    },
    "Bar & Snacks": {
        "Bar & Snacks": "https://www.myprotein.com.sg/dietary-needs/vegan/snacks.list",
        "Vegan Protein Bars & Cookies": "https://www.myprotein.com.sg/ranges/myvegan/vegan-protein-bars-cookies.list",
        "Vegan Superfoods": "https://www.myprotein.com.sg/dietary-needs/vegan/snacks/superfoods.list"
    },
    "Sports Nutrition": {
        "Sports Nutrition": "https://www.myprotein.com.sg/nutrition/vegan/sports-nutrition.list",
        "Vegan Creatine": "https://www.myprotein.com.sg/nutrition/creatine/vegan-creatine.list",
        "Vegan EAA": "https://www.myprotein.com.sg/sports-nutrition/impact-eaa/11985042.html"
    },
    "Wellbeing": {
        "Wellbeing": "https://www.myprotein.com.sg/ranges/myvitamins/wellbeing.list"
    },
    "Beauty": {
        "Beauty": "https://www.myprotein.com.sg/ranges/myvitamins/beauty.list"
    },
    "Collagen": {
        "Collagen": "https://www.myprotein.com.sg/elysium.search?search=collagen"
    }
}

# Insert URLs into the database
for category, subcategories in categories_urls.items():
    for subcategory, url in subcategories.items():
        try:
            data = {
                'category': category,
                'subcategory': subcategory,
                'url': url
            }
            response = supabase.table('category_urls').insert(data).execute()
            if response.get('status') != 201:  # Checking for success
                print(f"Failed to insert {subcategory} under {category}: {response.get('data')}")
        except Exception as e:
            print(f"Error inserting {subcategory} under {category}: {str(e)}")



Error inserting Protein Powders under Protein Powders: {'code': '23505', 'details': 'Key (category, subcategory)=(Protein Powders, Protein Powders) already exists.', 'hint': None, 'message': 'duplicate key value violates unique constraint "category_urls_category_subcategory_key"'}
Error inserting All Protein under Protein Powders: {'code': '23505', 'details': 'Key (category, subcategory)=(Protein Powders, All Protein) already exists.', 'hint': None, 'message': 'duplicate key value violates unique constraint "category_urls_category_subcategory_key"'}
Error inserting Whey Protein under Protein Powders: {'code': '23505', 'details': 'Key (category, subcategory)=(Protein Powders, Whey Protein) already exists.', 'hint': None, 'message': 'duplicate key value violates unique constraint "category_urls_category_subcategory_key"'}
Error inserting Collagen Protein under Protein Powders: {'code': '23505', 'details': 'Key (category, subcategory)=(Protein Powders, Collagen Protein) already exists.', 

In [20]:
import requests
from bs4 import BeautifulSoup

# URL to scrape
url = 'https://www.myprotein.com.sg/nutrition/protein.list?pageNumber=4'  # Change this to the desired URL

# Fetch the HTML content from the website
response = requests.get(url)
html_content = response.text

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Check for the "Use our search tips & try again" message
def check_out_of_bounds(soup):
    message = soup.find(text="Use our search tips & try again")
    return message is not None

# Check if the page is out of bounds
if check_out_of_bounds(soup):
    print("Page number out of bounds. Stopping scraping.")
else:
    print("Page is valid. Proceeding with scraping.")
    # Place your scraping code here


Page number out of bounds. Stopping scraping.


C:\Users\gunda\AppData\Local\Temp\ipykernel_22828\955023279.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  message = soup.find(text="Use our search tips & try again")
